### Login

Para poder subir el resultado del entrenamiento al hub debemos logearnos primero, para ello necesitamos un token

Para crear un token hay que ir a la página de [setings/tokens](https://huggingface.co/settings/tokens) de nuestra cuenta, nos aparecerá algo así

![User-Access-Token-dark](http://maximofn.com/wp-content/uploads/2024/03/User-Access-Token-dark.png)

Le damos a `New token` y nos aparecerá una ventana para crear un nuevo token

![new-token-dark](http://maximofn.com/wp-content/uploads/2024/03/new-token-dark.png)

Le damos un nombre al token y lo creamos con el rol `write`, o con el rol `Fine-grained`, que nos permite seleccionar exactamente qué permisos tendrá el token

Una vez creado lo copiamos y lo pegamos a continuación

In [1]:
# from huggingface_hub import notebook_login
# notebook_login()

### Dataset

Vamos a usar un dataset de [chistes en inglés](https://huggingface.co/datasets/Maximofn/short-jokes-dataset)

In [2]:
from datasets import load_dataset

jokes = load_dataset("Maximofn/short-jokes-dataset")
jokes

DatasetDict({
    train: Dataset({
        features: ['ID', 'Joke'],
        num_rows: 231657
    })
})

Vamos a verlo un poco

In [3]:
jokes

DatasetDict({
    train: Dataset({
        features: ['ID', 'Joke'],
        num_rows: 231657
    })
})

Vemos que es un único set de entrenamiento de más de 200 mil chistes. Así que más adelante lo tendremos que dividir en train y evaluación

Vamos a ver una muestra

In [4]:
from random import randint

idx = randint(0, len(jokes['train']) - 1)
jokes['train'][idx]

{'ID': 205438, 'Joke': 'A guy walks into a bar ouch'}

Vemos que tiene una ID del chiste que no nos interesa para nada y el propio chiste

Por si tienes poca memoria voy a hacer un subset del dataset, elije el porcentaje de chistes que quieres usar

In [5]:
percent_of_train_dataset = 0.001    # If you want 50% of the dataset, set this to 0.5

subset_dataset = jokes["train"].select(range(int(len(jokes["train"]) * percent_of_train_dataset)))
subset_dataset

Dataset({
    features: ['ID', 'Joke'],
    num_rows: 231
})

Ahora dividimos el subset en un conjunto de entrenamiento y otro de validación

In [6]:
percent_of_train_dataset = 0.90

split_dataset = subset_dataset.train_test_split(train_size=int(subset_dataset.num_rows * percent_of_train_dataset), seed=19, shuffle=False)
train_dataset = split_dataset["train"]
validation_test_dataset = split_dataset["test"]

split_dataset = validation_test_dataset.train_test_split(train_size=int(validation_test_dataset.num_rows * 0.5), seed=19, shuffle=False)
validation_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(f"Size of the train set: {len(train_dataset)}. Size of the validation set: {len(validation_dataset)}. Size of the test set: {len(test_dataset)}")

Size of the train set: 207. Size of the validation set: 12. Size of the test set: 12


### Tokenizador

Instanciamos el tokenizador

In [7]:
from transformers import AutoTokenizer

checkpoints = "openai-community/gpt2"

tokenizer = AutoTokenizer.from_pretrained(checkpoints)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Ahora ya hemos asignado el token de pading al token de end of string para que no nos de un error como antes

Vamos a añadir dos nuevos tokens de inicio de chiste y final de chiste para tener más control

In [8]:
new_tokens = ['<SJ>', '<EJ>']   # Start and end of joke tokens

num_added_tokens = tokenizer.add_tokens(new_tokens)
print(f"Added {num_added_tokens} tokens")

Added 2 tokens


Creamos una función para añadir los nuevos tokens a las sentencias

In [9]:
joke_column = "Joke"

def format_joke(example):
    example[joke_column] = '<SJ> ' + example['Joke'] + ' <EJ>'
    return example

Seleccionamos las columnas que no necesitamos

In [10]:
remove_columns = [column for column in train_dataset.column_names if column != joke_column]
remove_columns

['ID']

Formateamos el dataset y eliminamos las columnas que no necesitamos

In [11]:
train_dataset = train_dataset.map(format_joke, remove_columns=remove_columns)
validation_dataset = validation_dataset.map(format_joke, remove_columns=remove_columns)
test_dataset = test_dataset.map(format_joke, remove_columns=remove_columns)
subset_dataset, validation_dataset, test_dataset

(Dataset({
     features: ['ID', 'Joke'],
     num_rows: 231
 }),
 Dataset({
     features: ['Joke'],
     num_rows: 12
 }),
 Dataset({
     features: ['Joke'],
     num_rows: 12
 }))

Ahora creamos una función para tokenizar los chistes

In [12]:
def tokenize_function(examples):
    return tokenizer(examples[joke_column], padding="max_length", truncation=True, max_length=768, return_tensors="pt")

Tokenizamos el dataset y eliminamos la columna con el texto

In [13]:
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=[joke_column])
validation_dataset = validation_dataset.map(tokenize_function, batched=True, remove_columns=[joke_column])
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=[joke_column])
train_dataset, validation_dataset, test_dataset

(Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 207
 }),
 Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 12
 }),
 Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 12
 }))

### Modelo

Ahora instanciamos el modelo para generación de texto y le asignamos el token de pading al token de end of string

In [14]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(checkpoints)
model.config.pad_token_id = model.config.eos_token_id

Vemos el tamaño del vocabulario del modelo

In [15]:
vocab_size = model.config.vocab_size
vocab_size

50257

Tiene 50257 tokens, que es el tamaño del vocabulario de GPT2. Pero como hemos dicho que íbamos a crear dos tokens nuevos con el inicio de chiste y el final de chiste, los añadimos al modelo

In [16]:
model.resize_token_embeddings(len(tokenizer))

new_vocab_size = model.config.vocab_size
print(f"Old vocab size: {vocab_size}. New vocab size: {new_vocab_size}. Added {new_vocab_size - vocab_size} tokens")

Old vocab size: 50257. New vocab size: 50259. Added 2 tokens


Se han añadido los dos nuevos tokens

### Entrenamiento

Configuramos los parámetros de entrenamiento

In [17]:
from transformers import TrainingArguments

metric_name = "accuracy"
model_name = "GPT2-small-finetuned-Maximofn-short-jokes-dataset-casualLM"
output_dir = f"./training_results"
LR = 2e-5
BS_TRAIN = 1
BS_EVAL = 8
EPOCHS = 3
WEIGHT_DECAY = 0.01
WARMUP_STEPS = 100

training_args = TrainingArguments(
    model_name,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BS_TRAIN,
    per_device_eval_batch_size=BS_EVAL,
    warmup_steps=WARMUP_STEPS,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    lr_scheduler_type="cosine",
    warmup_ratio = 0.1,
    fp16=True,
    load_best_model_at_end=True,
    # metric_for_best_model=metric_name,
    # push_to_hub=True,
    # report_to="tensorboard",
)

Definimos el trainer

In [18]:
from transformers import Trainer, DataCollatorForLanguageModeling

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    # tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

In [19]:
# %load_ext tensorboard
# %tensorboard --logdir ./GPT2-small-finetuned-Maximofn-short-jokes-dataset-casualLM

Entrenamos

In [20]:
trainer.train()

  0%|          | 0/621 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Evaluación

Una vez entrenado evaluamos el modelo sobre el dataset de test

In [ ]:
trainer.evaluate(eval_dataset=test_dataset)

AttributeError: 'NoneType' object has no attribute 'get'

### Publicar el modelo

Creamos la model card

In [ ]:
# trainer.create_model_card()

KeyboardInterrupt: 

Lo publicamos

In [ ]:
# trainer.push_to_hub()

### Uso del modelo

Limpiamos todo lo posible

In [ ]:
import torch
import gc


def clear_hardwares():
    torch.clear_autocast_cache()
    torch.cuda.ipc_collect()
    torch.cuda.empty_cache()
    gc.collect()


clear_hardwares()
clear_hardwares()

Descargamos el modelo y lo usamos para generar chistes

In [ ]:
from transformers import pipeline

user = "maximofn"
checkpoints = f"{user}/{model_name}"
task = "text-generation"
text_generator = pipeline(task, model=checkpoints, tokenizer=checkpoints)

In [ ]:
def generate_joke(prompt_text):
    text = f"<SJ> {prompt_text}"
    tokens = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**tokens, max_new_tokens=256, eos_token_id=tokenizer.encode("<EJ>")[-1])
    return tokenizer.decode(output[0], skip_special_tokens=False)        

In [ ]:
generate_joke("Why didn't the frog cross the road?")

Setting `pad_token_id` to `eos_token_id`:50258 for open-end generation.


"<SJ> Why didn't the frog cross the road? Because he was too big for it. <EJ>"